In [13]:
Packages <- c("modelsummary", "lmerTest", "arrow", "mgcv", "dplyr" , "effects", "regclass")
invisible(lapply(Packages, library, character.only = TRUE))

Loading required package: bestglm

Loading required package: leaps

Loading required package: VGAM

Loading required package: stats4

Loading required package: splines


Attaching package: ‘VGAM’


The following object is masked from ‘package:mgcv’:

    s


The following object is masked from ‘package:modelsummary’:

    Max


Loading required package: rpart

Loading required package: randomForest

randomForest 4.7-1.2

Type rfNews() to see new features/changes/bug fixes.


Attaching package: ‘randomForest’


The following object is masked from ‘package:dplyr’:

    combine


Important regclass change from 1.3:
All functions that had a . in the name now have an _
all.correlations -> all_correlations, cor.demo -> cor_demo, etc.




In [14]:
datafolder <- '/home/hoyinchan/blue/Data/data2022/'

In [15]:
loincs_cofounder <- c('LG6033-7', #AST       
                    'LG49755-8') #uric acid
loincs_dependents <- c('LG49864-8', #Ca
                     'LG11363-5', #Na
                     'LG49936-4', 'LG6657-3') #K

In [16]:
K_Bound <-  c(3.2, 4.4) # 2823-3
Na_Bound <- c(132, 144) # 2951-2
Ca_Bound <- c(8, 9.3) # 17861-6

In [17]:
path1 = paste0(datafolder,'confounder_regression.parquet')
df <- arrow::read_parquet(path1)

In [18]:
df <- df %>%  mutate_at(vars('if_confound_med_LNa', 'if_confound_med_HCa', 'if_confound_med_HK', 'if_confound_med_LK', 'low_K', 'high_K', 'low_Na', 'high_Na', 'low_Ca', 'high_Ca', 'ID2'), factor)

In [19]:
df2 <- df

In [37]:
lm_model <- lmer(AST ~ UA + LDH + (1 | ID2), data = df2)

summary(lm_model)

In [56]:
calculate_slope <- function(df, chem, Na_Bound, K_Bound, Ca_Bound, last_v=NULL, raw_v=NULL, med_v=NULL){
    df_filter <- df
    df_filter_con <- df
    if (chem == 'Na'){ 
        eqn <- sCr ~  (1|ID2) + Na
        eqn_con <- sCr ~  (1|ID2) + Na  
        
        if (last_v == 'L'){
            df_filter <- df_filter %>% filter(low_Na==1)
        }
        if (raw_v == 'L'){
            df_filter <- df_filter %>% filter(Na<=Na_Bound[1])
        }
        df_filter_con <- df_filter %>% filter(if_confound_med_LNa==0)
    }
    
    if (chem == 'Ca'){
        eqn <- sCr ~  (1|ID2) + Ca
        eqn_con <- sCr ~  (1|ID2) + Ca 
        if (last_v == 'H'){
            df_filter <- df_filter %>% filter(high_Ca==1)
        }
        if (raw_v == 'H'){
            df_filter <- df_filter %>% filter(Ca>=Ca_Bound[2])
        }
        df_filter_con <- df_filter %>% filter(if_confound_med_HCa==0)        
    }
    
    if (chem == 'K'){
        if ((last_v != 'L') | (raw_v != 'L')){
            eqn <- sCr ~  (1|ID2) + K
            eqn_con <- sCr ~  (1|ID2) + K + AST + UA + LDH            
#            eqn_con <- sCr ~  (1|ID2) + K + UA + AST
#            eqn_con <- sCr ~  (1|ID2) + K + UA            
        } else {
            eqn <- sCr ~  (1|ID2) + K
            eqn_con <- sCr ~  (1|ID2) + K
        }
        
        if (last_v == 'L'){
            df_filter <- df_filter %>% filter(low_K==1)
            df_filter_con <- df_filter %>% filter(if_confound_med_LK==0)                       
        }
        if (raw_v == 'L'){
            df_filter <- df_filter %>% filter(K<=K_Bound[1])
            df_filter_con <- df_filter %>% filter(if_confound_med_LK==0)           
        }
        
        if (last_v == 'H'){
            df_filter <- df_filter %>% filter(high_K==1)
            df_filter_con <- df_filter %>% filter(if_confound_med_HK==0)                       
        }
        if (raw_v == 'H'){
            df_filter <- df_filter %>% filter(K>=K_Bound[2])
            df_filter_con <- df_filter %>% filter(if_confound_med_HK==0)           
        }
    }
        
    x_con <- lmer(eqn_con, data=df_filter_con, REML = FALSE, control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B')))    
    x <-     lmer(eqn,     data=df_filter,     REML = FALSE, control = lmerControl(optimizer ='optimx', optCtrl=list(method='L-BFGS-B'))) 
#    x_con <- lmer(eqn_con, data=df_filter_con)    
#    x <-     lmer(eqn,     data=df_filter)
    
    print(summary(x_con))
    print(summary(x))
#    return (c(x_con, x))
    
#    slope_con <- fixef(x_con)[[paste0(chem,'_scaled')]]
#    slope     <- fixef(x)[[paste0(chem,'_scaled')]]    
#    p_con     <- summary(x_con)$coefficients[paste0(chem,'_scaled'), "Pr(>|t|)"]
#    p         <- summary(x)$coefficients[paste0(chem,'_scaled'), "Pr(>|t|)"]    
    
    slope_con <- fixef(x_con)[[chem]]
    slope     <- fixef(x)[[chem]]    
    p_con     <- summary(x_con)$coefficients[chem, "Pr(>|t|)"]
    p         <- summary(x)$coefficients[chem, "Pr(>|t|)"]    
    
    diff      <- (slope-slope_con)/slope*100
    
    return (c(round(c(slope, slope_con, p<=0.05, p_con<=0.05, diff),4), x, x_con))
}
        

In [51]:
results = calculate_slope(df2, 'K', Na_Bound, K_Bound, Ca_Bound, last_v='L', raw_v='A')    
results[1:5]

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: eqn_con
   Data: df_filter_con

REML criterion at convergence: -1207.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.9786 -0.4300 -0.0613  0.3594  4.9128 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID2      (Intercept) 0.09332  0.3055  
 Residual             0.01143  0.1069  
Number of obs: 1781, groups:  ID2, 574

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  4.482e-01  3.152e-02  1.705e+03  14.222  < 2e-16 ***
K            3.520e-02  8.125e-03  1.353e+03   4.333 1.58e-05 ***
AST         -1.571e-06  4.162e-05  7.396e+02  -0.038     0.97    
UA           4.576e-02  2.234e-03  1.435e+03  20.482  < 2e-16 ***
LDH          3.946e-05  8.108e-06  1.457e+03   4.867 1.26e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr) K      AST    UA    
K   -0.848    

[[1]]
[1] 0.085

[[2]]
[1] 0.0352

[[3]]
[1] 1

[[4]]
[1] 1

[[5]]
[1] 58.5897

In [53]:
results = calculate_slope(df2, 'K', Na_Bound, K_Bound, Ca_Bound, last_v='H', raw_v='A')    
results[1:5]

Warning message:
“Some predictor variables are on very different scales: consider rescaling”
Warning message:
“Some predictor variables are on very different scales: consider rescaling”


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: eqn_con
   Data: df_filter_con

REML criterion at convergence: 20007.2

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.703 -0.151 -0.058  0.037 35.737 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID2      (Intercept) 0.1539   0.3924  
 Residual             1.7039   1.3053  
Number of obs: 5801, groups:  ID2, 1733

Fixed effects:
              Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)  2.268e-01  1.698e-01  3.777e+03   1.335   0.1819    
K            7.776e-02  3.908e-02  4.621e+03   1.990   0.0467 *  
AST         -9.400e-05  1.315e-04  4.482e+03  -0.715   0.4747    
UA           8.651e-02  9.652e-03  2.485e+03   8.962   <2e-16 ***
LDH         -2.708e-05  3.353e-05  2.105e+03  -0.808   0.4194    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
    (Intr) K      AST    UA    
K   -0.947             

[[1]]
[1] 0.0274

[[2]]
[1] 0.0778

[[3]]
[1] 1

[[4]]
[1] 1

[[5]]
[1] -183.7124

In [57]:
results <- calculate_slope(df, 'Na', Na_Bound, K_Bound, Ca_Bound, last_v='L', raw_v='A')    
results[1:5]

Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: eqn_con
   Data: df_filter_con
Control: lmerControl(optimizer = "optimx", optCtrl = list(method = "L-BFGS-B"))

     AIC      BIC   logLik deviance df.resid 
 -8312.6  -8270.0   4160.3  -8320.6   311599 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-94.209  -0.351  -0.058   0.283 241.117 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID2      (Intercept) 0.16112  0.4014  
 Residual             0.03039  0.1743  
Number of obs: 311603, groups:  ID2, 65838

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept) 7.787e-01  1.427e-02 2.792e+05  54.577  < 2e-16 ***
Na          6.764e-04  1.083e-04 2.743e+05   6.245 4.24e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
   (Intr)
Na -0.994
Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  

[[1]]
[1] 2e-04

[[2]]
[1] 7e-04

[[3]]
[1] 0

[[4]]
[1] 1

[[5]]
[1] -319.6687

In [58]:
results <- calculate_slope(df, 'Ca', Na_Bound, K_Bound, Ca_Bound, last_v='H', raw_v='A')    
results[1:5]

Linear mixed model fit by maximum likelihood . t-tests use Satterthwaite's
  method [lmerModLmerTest]
Formula: eqn_con
   Data: df_filter_con
Control: lmerControl(optimizer = "optimx", optCtrl = list(method = "L-BFGS-B"))

      AIC       BIC    logLik  deviance  df.resid 
-232207.2 -232160.4  116107.6 -232215.2    885331 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-53.510  -0.382  -0.059   0.310 289.575 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID2      (Intercept) 0.15020  0.3876  
 Residual             0.02325  0.1525  
Number of obs: 885335, groups:  ID2, 189796

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept) 6.416e-01  3.736e-03 8.228e+05  171.75   <2e-16 ***
Ca          3.290e-02  3.828e-04 7.528e+05   85.95   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
   (Intr)
Ca -0.969
Linear mixed model fit by maximum likelihood . t-tests use Sattert

[[1]]
[1] 0.0332

[[2]]
[1] 0.0329

[[3]]
[1] 1

[[4]]
[1] 1

[[5]]
[1] 0.8754